## Notes

* asterisk indicates suppressed value (see below)
  * 2,788 suppressed values in 2020-2021 school year
  * Impute median of similar-sized schools for these?

"* indicates cell has been suppressed, a blank cell indicates no report has been received for a given district in the indicated time period, a 0 indicates that a report was received and no cases were reported for that group in the reported time period. Single-campus student cases and sources of infection are suppressed when (1) reported student cases are fewer than 5, (2) a campus has at least a 90% student positivity rate when on-campus enrollment for a school is at least 15 students, or (3) a campus has at least a 50% positivity rate when on-campus enrollment has fewer than 15 students. If only one campus in a district has suppressed student numbers then student and source of infection numbers for the campus with the next smallest numbers of positive students are also suppressed. Cumulative student cases and sources of infection numbers for a campus are suppressed when (1) student cases are less than five, or (2) current report numbers have been suppressed for the first three weeks that student cases are reported. If there is only one campus reporting in a district and it is a multiple campus, student and source of infection numbers are not suppressed for the district total. Otherwise, district totals are suppressed when (1) student cases are fewer than 5, or (2) a district has at least a 90% student positivity rate when total district enrollment is least 15 students, or, (3) a district has at least a 50% positivity rate when total district enrollment has fewer than 15 students, or (4) cases on a campus have been suppressed for the first three weeks that student cases are reported and there are fewer than 5 campuses reporting in a district."

In [1]:
import pandas as pd
import numpy as np
import missingno as msno

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# Loading Excel file to get sheet names

sheets1 = pd.ExcelFile('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls')
print(sheets1.sheet_names)

# Loading one dataframe to get column names

nov22 = pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', sheet_name="Campus Report_November 22", header=5)
print(nov22.head(20))

In [2]:
# List of sheet names from original Excel file (excludes district reports)

sheets = ['Campus Report_August 1', 'Campus Report_July 25', 'Campus Report_July 18', 'Campus Report_July 11', 
          'Campus Report_July 04', 'Campus Report_June 27', 'Campus Report_June 20', 'Campus Report_June13', 
          'Campus Report_June 6', 'Campus Report_May 30', 'Campus Report_May 23', 'Campus Report_May 16', 
          'Campus Report_May 09', 'Campus Report_May 02', 'Campus Report_April 25', 'Campus Report_April 18', 
          'Campus Report_April 11', 'Campus Report_April 4', 'Campus Report_March 28', 'Campus Report_March 21', 
          'Campus Report_March 14', 'Campus Report_March 7', 'Campus Report_February 28', 
          'Campus Report_February 21', 'Campus Report_February 14', 'Campus Report_February 7', 
          'Campus Report_January 31', 'Campus Report_January 24', 'Campus Report_January 17', 
          'Campus Report_January 12', 'Campus Report_January 5th', 'Campus Report_December 29', 
          'Campus Report_December 20', 'Campus Report_December 15', 'Campus Report_December 6', 
          'Campus Report_November 29']

# Reversing list to get reports in order from beginning of school year

sheets.reverse()

# Getting first weekly report as base dataframe

df = pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', sheet_name="Campus Report_November 22", header=5)

# Renaming columns that will be kept (at least at first --
# I'm sure I'll remove many of them once I figure out the exact analyses)

df.rename(columns = {'District Name':'District', 'District\nLEA\nNumber':'Dist LEA', 
                    'Total District\nEnrollment as\nof October 30, 2020':'Dist Enrollment 10/30/20', 
                    'Approximate\nDistrict On Campus\nEnrollment as of\nOctober 30, 2020':'Dist On-Campus Enrollment 10/30/20', 
                    'Total School\nEnrollment as\nof October 30, 2020':'Sch Enrollment 10/30/2020', 
                    'On-Campus\nEnrollment for\nSchool as of\nOctober 30, 2020':'Sch On-Campus Enrollment 10/30/2020', 
                    'Campus\nID':'Campus ID', 'Total\nStudent\nCases':'Total student cases_Nov22', 
                    'Total\nStaff\nCases':'Total staff cases_Nov22'},
                     inplace=True)

# Empty list for adding abbreviations (to be appended to column names)

abbrevs=[]

# List for raw dataframes

ds = []

for i, sheet in enumerate(sheets):
    
    # Loading sheets into dataframes (df2–df36)
    # Starts with df2 because of base dataframe above
    
    globals()['d' + str(i + 2)] = pd.read_excel('../data/raw/Public-School-Data-Files-2020_2021-School-Year.xls', 
                                                sheet_name = sheet, header=5)
    
    # Renaming student and staff total columns (used columns.values becase not all
    # sheets use the same column headers)
    
    globals()['d' + str(i+2)].rename(columns = {globals()['d' + str(i+2)].columns.values[13]: 
                                                          'Total student cases', 
                                                globals()['d' + str(i+2)].columns.values[14]: 
                                                            'Total staff cases'}, inplace=True)
    
    # Add name of raw dataframe to ds list for iterating over later
    
    ds.append(str('d' + str(i + 2)))
    
    # Create abbreviations for appending to columns in final merged dataframe
    
    if sheet[-2] == " ":
        abbrev = sheet[14:17] + sheet[-1]
    else:
        abbrev = sheet[14:17] + sheet[-2:]
        
    if abbrev == 'Janth':
        abbrev = "Jan5" 
    
    # Add abbreviations to list
    
    abbrevs.append(abbrev)

In [3]:
# List for dataframes with only columns of interest

dfs = []

for i, d in enumerate(ds):
    
    # Create empty dataframe to append the updated columns from raw df
    
    globals()['df' + str(i+2)] = pd.DataFrame()
    
    # Replace 'multiple campus' listing with NaNs
    # Add to new dataframe as float (float instead of int because it's nullable)
    
    globals()[d]['Campus\nID'].replace('Multiple\nCampus', np.nan, inplace=True) 
    globals()['df' + str(i+2)]['Campus ID'] = globals()[d]['Campus\nID'].str.strip("'").astype('float')
    
    # Convert asterisks (suppressed values) and blank spaces to NaN, add to new dataframe
    # Using float again so NaNs can coexist in column
    
    globals()[d]['Total student cases'].replace([r'*', ' '], np.nan, inplace=True)
    globals()[d]['Total staff cases'].replace([r'*', ' '], np.nan, inplace=True)
    
    globals()['df' + str(i+2)]['Total student cases_' + abbrevs[i]] = globals()[d]['Total student cases'].astype('float')
    globals()['df' + str(i+2)]['Total staff cases_' + abbrevs[i]] = globals()[d]['Total staff cases'].astype('float')
    
    # Add name of new dataframe to dfs list
    
    dfs.append(str('df' + str(i+2)))

In [4]:
# Replace 'multiple campus' listing in base dataframe with NaN
# Update base dataframe dtypes to match

df['Campus ID'].replace('Multiple\nCampus', np.nan, inplace=True) 
df['Campus ID'] = df['Campus ID'].str.strip("'").astype('float')
    
# Convert asterisks (suppressed values) to NaN, add to new dataframe
# Using float again so NaNs can coexist in column
    
df['Total student cases_Nov22'].replace([r'*', ' '], np.nan, inplace=True)
df['Total staff cases_Nov22'].replace([r'*', ' '], np.nan, inplace=True)
    
df['Total student cases_Nov22'] = df['Total student cases_Nov22'].astype('float')
df['Total staff cases_Nov22'] = df['Total staff cases_Nov22'].astype('float')

In [5]:
# Drop rows where there is no campus ID
# Loop for dataframes in dfs list, base dataframe after

for i in dfs:
    
    globals()[i].dropna(subset='Campus ID', inplace=True)
    globals()[i]['Campus ID'] = globals()[i]['Campus ID'].astype(int)
    
df.dropna(subset='Campus ID', inplace=True)
df['Campus ID'] = df['Campus ID'].astype(int)

In [6]:
# One campus has duplicated reports (can detail this in report later),
# and it looks like like the second report is more likely to be accurate.

for i in dfs:
    
    globals()[i].drop_duplicates(subset='Campus ID', keep='last', inplace=True)

In [7]:
for i in dfs:
    
    # Merge all dataframes
    
    df = df.merge(globals()[i], how='inner', on='Campus ID')

In [8]:
df.columns

Index(['District', 'Dist LEA', 'Dist Enrollment 10/30/20',
       'Dist On-Campus Enrollment 10/30/20', 'Campus Name', 'Campus ID',
       'Sch Enrollment 10/30/2020', 'Sch On-Campus Enrollment 10/30/2020',
       'New\nStudent\nCases', 'New\nStaff\nCases', 'On\nCampus', 'Off\nCampus',
       'Unknown', 'Total student cases_Nov22', 'Total staff cases_Nov22',
       'On\nCampus.1', 'Off\nCampus.1', 'Unknown.1',
       'Total student cases_Nov29', 'Total staff cases_Nov29',
       'Total student cases_Dec6', 'Total staff cases_Dec6',
       'Total student cases_Dec15', 'Total staff cases_Dec15',
       'Total student cases_Dec20', 'Total staff cases_Dec20',
       'Total student cases_Dec29', 'Total staff cases_Dec29',
       'Total student cases_Jan5', 'Total staff cases_Jan5',
       'Total student cases_Jan12', 'Total staff cases_Jan12',
       'Total student cases_Jan17', 'Total staff cases_Jan17',
       'Total student cases_Jan24', 'Total staff cases_Jan24',
       'Total student c

In [9]:
# List of dates for looking at all reports, just student reports, and
# just staff reports.

all_dates = ['Total student cases_Nov22', 'Total staff cases_Nov29', 'Total student cases_Nov29', 
             'Total staff cases_Nov29', 'Total student cases_Dec6', 'Total staff cases_Dec6', 
             'Total student cases_Dec15', 'Total staff cases_Dec15', 'Total student cases_Dec20', 
             'Total staff cases_Dec20', 'Total student cases_Dec29', 'Total staff cases_Dec29', 
             'Total student cases_Jan5', 'Total staff cases_Jan5', 'Total student cases_Jan12', 
             'Total staff cases_Jan12', 'Total student cases_Jan17', 'Total staff cases_Jan17', 
             'Total student cases_Jan24', 'Total staff cases_Jan24', 'Total student cases_Jan31', 
             'Total staff cases_Jan31', 'Total student cases_Feb7', 'Total staff cases_Feb7', 
             'Total student cases_Feb14', 'Total staff cases_Feb14', 'Total student cases_Feb21', 
             'Total staff cases_Feb21', 'Total student cases_Feb28', 'Total staff cases_Feb28', 
             'Total student cases_Mar7', 'Total staff cases_Mar7', 'Total student cases_Mar14', 
             'Total staff cases_Mar14', 'Total student cases_Mar21', 'Total staff cases_Mar21', 
             'Total student cases_Mar28', 'Total staff cases_Mar28', 'Total student cases_Apr4', 
             'Total staff cases_Apr4', 'Total student cases_Apr11', 'Total staff cases_Apr11', 
             'Total student cases_Apr18', 'Total staff cases_Apr18', 'Total student cases_Apr25', 
             'Total staff cases_Apr25', 'Total student cases_May02', 'Total staff cases_May02', 
             'Total student cases_May09', 'Total staff cases_May09', 'Total student cases_May16', 
             'Total staff cases_May16', 'Total student cases_May23', 'Total staff cases_May23', 
             'Total student cases_May30', 'Total staff cases_May30', 'Total student cases_Jun6', 
             'Total staff cases_Jun6', 'Total student cases_Jun13', 'Total staff cases_Jun13', 
             'Total student cases_Jun20', 'Total staff cases_Jun20', 'Total student cases_Jun27', 
             'Total staff cases_Jun27', 'Total student cases_Jul04', 'Total staff cases_Jul04', 
             'Total student cases_Jul11', 'Total staff cases_Jul11', 'Total student cases_Jul18', 
             'Total staff cases_Jul18', 'Total student cases_Jul25', 'Total staff cases_Jul25', 
             'Total student cases_Aug1', 'Total staff cases_Aug1']

student_dates = ['Total student cases_Nov22', 'Total student cases_Nov29', 'Total student cases_Dec6', 
       'Total student cases_Dec15', 'Total student cases_Dec20', 'Total student cases_Dec29', 
       'Total student cases_Jan5', 'Total student cases_Jan12', 'Total student cases_Jan17', 
       'Total student cases_Jan24', 'Total student cases_Jan31', 'Total student cases_Feb7', 
       'Total student cases_Feb14', 'Total student cases_Feb21', 'Total student cases_Feb28', 
       'Total student cases_Mar7', 'Total student cases_Mar14', 'Total student cases_Mar21', 
       'Total student cases_Mar28', 'Total student cases_Apr4', 'Total student cases_Apr11', 
       'Total student cases_Apr18', 'Total student cases_Apr25', 'Total student cases_May02', 
       'Total student cases_May09', 'Total student cases_May16', 'Total student cases_May23', 
       'Total student cases_May30', 'Total student cases_Jun6', 'Total student cases_Jun13', 
       'Total student cases_Jun20', 'Total student cases_Jun27', 'Total student cases_Jul04', 
       'Total student cases_Jul11', 'Total student cases_Jul18', 'Total student cases_Jul25', 
       'Total student cases_Aug1']

staff_dates = ['Total staff cases_Nov22', 'Total staff cases_Nov29', 'Total staff cases_Dec6', 
       'Total staff cases_Dec15', 'Total staff cases_Dec20', 'Total staff cases_Dec29', 
       'Total staff cases_Jan5', 'Total staff cases_Jan12', 'Total staff cases_Jan17', 
       'Total staff cases_Jan24', 'Total staff cases_Jan31', 'Total staff cases_Feb7', 
       'Total staff cases_Feb14', 'Total staff cases_Feb21', 'Total staff cases_Feb28', 
       'Total staff cases_Mar7', 'Total staff cases_Mar14', 'Total staff cases_Mar21', 
       'Total staff cases_Mar28', 'Total staff cases_Apr4', 'Total staff cases_Apr11', 
       'Total staff cases_Apr18', 'Total staff cases_Apr25', 'Total staff cases_May02', 
       'Total staff cases_May09', 'Total staff cases_May16', 'Total staff cases_May23', 
       'Total staff cases_May30', 'Total staff cases_Jun6', 'Total staff cases_Jun13', 
       'Total staff cases_Jun20', 'Total staff cases_Jun27', 'Total staff cases_Jul04', 
       'Total staff cases_Jul11', 'Total staff cases_Jul18', 'Total staff cases_Jul25', 
       'Total staff cases_Aug1']

In [10]:
# Percentage of missing data from student report columns
# Total number of reports from student columns

pd.set_option('display.max_rows', None)

print(df[student_dates].isna().sum()/len(df[student_dates]))
print(df[student_dates].notnull().sum())

Total student cases_Nov22    0.693511
Total student cases_Nov29    0.666277
Total student cases_Dec6     0.645745
Total student cases_Dec15    0.641064
Total student cases_Dec20    0.616702
Total student cases_Dec29    0.611170
Total student cases_Jan5     0.608830
Total student cases_Jan12    0.576064
Total student cases_Jan17    0.622872
Total student cases_Jan24    0.502447
Total student cases_Jan31    0.462872
Total student cases_Feb7     0.450957
Total student cases_Feb14    0.442021
Total student cases_Feb21    0.436915
Total student cases_Feb28    0.435106
Total student cases_Mar7     0.429468
Total student cases_Mar14    0.426809
Total student cases_Mar21    0.424894
Total student cases_Mar28    0.420213
Total student cases_Apr4     0.417660
Total student cases_Apr11    0.414043
Total student cases_Apr18    0.409362
Total student cases_Apr25    0.405957
Total student cases_May02    0.402553
Total student cases_May09    0.399787
Total student cases_May16    0.397021
Total studen